## Testing Twitter functions

In [7]:
import twitter
from twitter import Tweet
import google_api
import time
from datetime import datetime, timedelta

In [9]:
now = datetime.today()
to_gmt = timedelta(4/24-(1/60)/24)
now = now + to_gmt
past = timedelta(0.5/24)
new_date = now - past

print(now.strftime('%Y-%m-%dT%H:%M:%S.000Z'))
print(new_date.strftime('%Y-%m-%dT%H:%M:%S.000Z'))

2021-10-10T04:52:04.000Z
2021-10-10T04:22:04.000Z


In [7]:

now = datetime.today()
print(now)

days = timedelta(0.25/24)
new_date = now - days
print(new_date.strftime('%Y-%m-%dT%H:%M:%S.000Z'))

#print(datetime.today().strftime('%Y-%m-%dT%H:%M:%S.000Z'))

2021-10-09 12:11:49.146919
2021-10-09T11:56:49.000Z


In [3]:
print(twitter.get_date(10))

2021-09-28T21:17:54.000Z


In [4]:
headers = twitter.create_headers()
url = twitter.create_url("Donald Trump")
print(url)
parameters = twitter.create_params(max_results=100, start_time=twitter.get_date(1))
tweets = twitter.connect_to_endpoint_twitter(url, headers, parameters)


https://api.twitter.com/2/tweets/search/recent?query=Donald%20Trump%20lang%3Aen%20-is%3Aretweet
Endpoint Response Code: 200


In [9]:
print(len(tweets['data']))

#twitter.print_tweet_dump(tweets)
print(tweets['meta']['next_token'])
print(tweets['data'][0]['created_at'])
print(tweets['data'][-1]['created_at'])

10
2021-10-09T00:26:29.000Z
2021-10-09T00:26:00.000Z


## Testing using next token to get more tweets

In [5]:
print("Last tweet at: {}".format(tweets['data'][-1]['created_at']))
tweet_count = len(tweets['data'])
next = tweets['meta']['next_token']

while next != None:
    tweets = twitter.connect_to_endpoint_twitter(url, headers, parameters, next_token=next)
    print("Response recieved with {} tweets. Last tweet at: {}".format(len(tweets['data']), tweets['data'][-1]['created_at']))
    tweet_count += len(tweets['data'])
    print("tweet count: " + str(tweet_count))
    try:
        next = tweets['meta']['next_token']
    except KeyError:
        next = None

Last tweet at: 2021-10-09T01:08:57.000Z
Endpoint Response Code: 200
Response recieved with 100 tweets. Last tweet at: 2021-10-09T01:01:05.000Z
tweet count: 200
Endpoint Response Code: 200
Response recieved with 100 tweets. Last tweet at: 2021-10-09T00:51:59.000Z
tweet count: 300
Endpoint Response Code: 200
Response recieved with 100 tweets. Last tweet at: 2021-10-09T00:44:15.000Z
tweet count: 400
Endpoint Response Code: 200
Response recieved with 100 tweets. Last tweet at: 2021-10-09T00:35:48.000Z
tweet count: 500
Endpoint Response Code: 200
Response recieved with 99 tweets. Last tweet at: 2021-10-09T00:27:18.000Z
tweet count: 599
Endpoint Response Code: 200
Response recieved with 100 tweets. Last tweet at: 2021-10-09T00:19:40.000Z
tweet count: 699
Endpoint Response Code: 200
Response recieved with 100 tweets. Last tweet at: 2021-10-09T00:09:29.000Z
tweet count: 799
Endpoint Response Code: 200
Response recieved with 100 tweets. Last tweet at: 2021-10-09T00:01:04.000Z
tweet count: 899
E

KeyboardInterrupt: 

In [8]:
print(tweets['meta'])

{'newest_id': '1446574021307510789', 'oldest_id': '1446572736562802690', 'result_count': 59}


## Test Google functions

In [7]:
google_client = google_api.get_google_client()
neut_count = 0
pos_count = 0
neg_count = 0

for tweet in tweets['data']:
    tweet_ob = Tweet(tweet)
    tweet_ob.clean_tweet()
    text, feeling, score = google_api.analyze_tweet(tweet_ob.text, google_client)
    if feeling == 'positive':
        pos_count += 1
    elif feeling == 'negative':
        neg_count += 1
    else:
        neut_count +=1

    print(text)
    print(feeling + " (" + str(score) + ")")
    print()

print("Positive count {} Negative count {} Neutral count {}".format(str(pos_count), str(neg_count), str(neut_count)))


She makes AOC look smart. 
positive (0.8999999761581421)

@PhilipZimmer4 @Yeahthatsrigh18 @arsheffey @AOC @Ilhan @SecCardona This argument relies on the idea that ppl want to live off social safety net programs their entire lives, an idea that's been disproven ad nauseam both in data and anecdotally—and is often perpetuated by racists to explain demographic relationships that are corollary not causal.
negative (-0.8999999761581421)

@avivnator @m7mdkurd @AOC The Arabs invaded in 636, not 1187. Get your facts straight if you’re gonna be a pedant.
negative (-0.6000000238418579)

Hear me out AOC only fans will clear out the national debt by lunch 
negative (-0.30000001192092896)

#RedPillBill 👉🐿️⛰️❤️ #ThePsychicTicker #CriswellNews @RealJamesWoods @SenSchumer @AOC @SpeakerPelosi @BarackObama @HillaryClinton @BillClinton @boderek @JoeBiden @KamalaHarris @MeetThePress  "This time we got the gas."  Oh yeah.
positive (0.4000000059604645)

@TheJusticeDept @DHSgov 
@SenateDems @thejointstaff @A